In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Colab Notebooks/딥러닝

/content/drive/MyDrive/Colab Notebooks/딥러닝


In [3]:
import torch

### 파이썬으로 RNN 구현하기

```
# 아래의 코드는 의사 코드(pseudocode)로 실제 동작하는 코드가 아님.

hidden_state_t = 0 # 초기 은닉 상태를 0(벡터)로 초기화
for input_t in input_length: # 각 시점마다 입력을 받는다.
     output_t = tanh(input_t, hidden_state_t) # 각 시점에 대해서 입력과 은닉 상태를 가지고 연산
     hidden_state_t = output_t # 연산 결과를 다음 시점의 은닉 상태로 사용
```

In [7]:
import numpy as np

timesteps = 10  # 시점의 수 -> NLP 분야에서는 보통 문장의 길이! (구성하는 단어의 수)
input_size = 4 # 단어 벡터 디멘진. 입력의 차원, NLP에서는 보통 단어 벡터의 차원이 된다.
hidden_size = 8 # 은닉 상태의 크기 -> 메모리 셀의 용량

inputs = np.random.random((timesteps, input_size))  # 입력에 해당되는 2D 텐서

hidden_state_t = np.zeros((hidden_size,))  # 초기 은닉 상태는 0(벡터)로 초기화
# 은닉 상태의 크기 hidden_size로 은닉 상태를 만듦.

- 시점, 입력의 차원, 은닉 상태의 크기, 그리고 초기 은닉 상태를 정의한 상태
- 현재 초기 은닉 상태는 0의 값을 가지는 벡터로 초기화가 된 상태

In [8]:
print(hidden_state_t)  # 8의 크기를 가지는 은닉 상태. 현재는 초기 은닉 상태로 모든 차원이 9의 값을 가짐.

[0. 0. 0. 0. 0. 0. 0. 0.]


- 은닉 상태의 크기를 8로 정의하였으므로 8의 차원을 가지는 0의 값으로 구성된 벡터가 출력

In [9]:
Wx = np.random.random((hidden_size, input_size))  # 입력에 대한 가중치 행렬
Wh = np.random.random((hidden_size, hidden_size))  # 은닉 상태에 대한 가중치 행렬
b = np.random.random((hidden_size,))  # 편향(bias)

- 가중치와 편향을 각 크기에 맞게 정의하였다.

In [12]:
print(np.shape(Wx))  # 은닉 상태의 크기 * 입력의 차원
print(np.shape(Wh)) # 은닉 상태의 크기 * 은닉 상태의 크기
print(np.shape(b)) # 은닉 상태의 크기

(8, 4)
(8, 8)
(8,)


In [11]:
total_hidden_states = []  # 은닉 상태를 저장할 리스트

# Memory cell의 동작 과정
for input_t in inputs:  # 각 시점에 따른 입력값
    # tanh(W_x * x_t + W_h * h(t-1) + b)
    output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)
    total_hidden_states.append(list(output_t))  # 각 시점의 은닉 상태 값을 계속해서 추적
    hidden_state_t = output_t  # 은닉층에서의 계산값을 다음 시점으로 전달

total_hidden_states = np.stack(total_hidden_states, axis=0)  # 리스트를 넘파이 배열로 변환. 세로로 쌓기 위함

print(total_hidden_states)

[[0.99950732 0.99999252 0.99984194 0.99945611 0.99986913 0.99993054
  0.99997892 0.99999027]
 [0.99964221 0.99999641 0.99994314 0.99940055 0.99992651 0.99995004
  0.99999007 0.99999457]
 [0.99944975 0.9999938  0.99988686 0.99932181 0.99986359 0.9998874
  0.99997946 0.99998925]
 [0.99979024 0.99999799 0.99997263 0.99977511 0.99994925 0.9999317
  0.99999564 0.99999731]
 [0.99924808 0.99998838 0.9997584  0.99936374 0.99977425 0.99978362
  0.99996008 0.99997859]
 [0.99957494 0.99999452 0.99991995 0.99937717 0.99991223 0.99991575
  0.99998537 0.99998962]
 [0.9997957  0.99999536 0.99994098 0.99974742 0.99995951 0.99996478
  0.99999328 0.99999387]
 [0.9998107  0.9999966  0.99997742 0.99961784 0.99997601 0.99993563
  0.99999523 0.99999013]
 [0.99974636 0.99999692 0.99995982 0.99964858 0.99994817 0.99994411
  0.99999348 0.99999522]
 [0.99960468 0.99999254 0.99990398 0.9997087  0.9998963  0.99974108
  0.99998374 0.99998346]]


### 파이토치의 nn.RNN()
- 파이토치에서는 nn.RNN()을 통해서 RNN 셀을 구현

In [13]:
import torch.nn as nn

In [14]:
input_size = 5  # 입력의 크기
hidden_size = 8  # 은닉 상태의 크기

In [15]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

In [16]:
cell = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)

In [17]:
outputs, _status = cell(inputs)

- RNN 셀은 두 개의 입력을 리턴하는데, 첫번째 리턴값은 모든 시점(timesteps)의 은닉 상태들이며, 두번째 리턴값은 마지막 시점(timestep)의 은닉 상태

In [18]:
print(outputs.shape)  # (batch_size, time_steps, hidden_size)
print(_status.shape)  # (num_layers * num_directions, batch_size, hidden_size

torch.Size([1, 10, 8])
torch.Size([1, 1, 8])


In [19]:
torch.Size([1, 10, 8])

torch.Size([1, 10, 8])

### 깊은 순환 신경망

In [20]:
# (batch_size, time_steps, input_size)
inputs = torch.Tensor(1, 10, 5)

cell = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)

In [21]:
print(outputs.shape)  # (batch_size, time_steps, hidden_size

torch.Size([1, 10, 8])
